## 信息增益

信息增益(information gain)表示得知特征X的信息而使得类Y的信息的不确定性减少的程度。特征A训练数据集D的信息増益$g(D,A)$，定义为集合D的经验熵$H(D)$与特征A给定条件下D的经验条件熵$H(D∣A)$之差，即

$g(D,A)=H(D)−H(D∣A)$

## 信息增益率

信息增益值的大小是相对于训练数据集而言的，并没有绝对意义。在分类问题困难时，也就是说在训练数据集的经验熵大的时候，信息增益值会偏大。反之，信息增益值会偏小。使用信息增益比(information gain ratio)可以对这一问题进行校正。这是特征选择的另一准则。

信息增益比：特征A对训练数据集D的信息增益比$g_R(D,A)$定义为其信息增益$g(D,A)$与训练数据集D的经验熵$H(D)$之比

## 1. ID3算法

1) 定义：ID3算法的核心是在决策树各个结点上应用[信息增益]准则选择特征，递归地构建决策树。

具体方法是：从根结点(root node)开始，对结点计算所有可能的特征的信息增益，选择信息增益最大的特征作为结点的特征，由该特征的不同取值建立子结点；再对子结点递归地调用以上方法，构建决策树。直到所有特征的信息增益均很小或没有特征可以选择为止，最后得到一个决策树。ID3 相当于用极大似然法进行概率模型的选择。

2）优点

- 理论清晰，算法简单，很有实用价值的示例学习算法；
- 计算时间是例子个数、特征属性个数、节点个数之积的线性函数，总预测准确率较令人满意。
3）缺点

- 存在偏向问题，各特征属性的取值个数会影响互信息量的大小；
- 特征属性间的相关性强调不够，是单变元算法；
- 对噪声较为敏感，训练数据的轻微错误会导致结果的不同；
- 结果随训练集记录个数的改变而不同，不便于进行渐进学习；
- 生成的树容易产生过拟合。

## 2. C4.5的生成算法
1）定义

C4.5算法与ID3算法相似，C4.5 算法对ID3算法进行了改进。C4.5 在生成的过程中，用信息增益比来选择特征。

2）优点

- 通过信息增益率选择分裂属性，克服了ID3算法中通过信息增益倾向于选择拥有多个属性值的属性作为分裂属性的不足；
- 能够处理离散型和连续型的属性类型，即将连续型的属性进行离散化处理；
- 构造决策树之后进行剪枝操作；
- 能够处理具有缺失属性值的训练数据。

3）缺点

- C4.5生成的是多叉树，即一个父节点可以有多个节点，因此算法的计算效率较低，特别是针对含有连续属性值的训练样本时表现的尤为突出。
- 算法在选择分裂属性时没有考虑到条件属性间的相关性，只计算数据集中每一个条件属性与决策属性之间的期望信息，有可能影响到属性选择的正确性。
- C4.5算法只适合于能够驻留内存的数据集，当训练集大得无法在内存容纳时，程序无法运行；


In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('resource/watermelon_3a.csv')

def calc_entropy(dataSet):
    m=len(dataSet)
    labelcounts={}
    for i in range(m):
        label=dataSet[i][-1]
        labelcounts[label]=labelcounts.get(label,0)+1
    entropy=0.0
    for counts in labelcounts.values():
        prob=counts/m
        entropy-=prob*np.log2(prob)
    return entropy

def splitDataSet(dataSet, axis, value):
    retdataSet=[]
    for data in dataSet:
        if data[axis]==value:
            subFeatures=data[:axis]
            subFeatures.extend(data[axis+1:])
            retdataSet.append(subFeatures)
    return retdataSet

def chooseBestFeatureToSplit(dataSet):
    feature_nums=len(dataSet[0])-1
    baseEntropy=calc_entropy(dataSet)
    best_infor_gain=0.0
    best_feature=-1
    for i in range(feature_nums):
        feature_list=[example[i] for example in dataSet]
        unique_value=set(feature_list)
        new_entropy=0.0
        for value in unique_value:
            subDataSet=splitDataSet(dataSet,i,value)
            prob=len(subDataSet)/len(dataSet)
            new_entropy+=prob*calc_entropy(subDataSet)
        infor_gain=baseEntropy-new_entropy
        if infor_gain>best_infor_gain:
            best_infor_gain=infor_gain
            best_feature=i
    return best_feature


def majorityCnt(classList):
    m=len(classList)
    class_nums={}
    for i in range(m):
        label=classList[i]
        class_nums[label]=class_nums.get(label,0)+1
    sorted_class_nums=sorted(class_nums.items(),key=lambda x:x[1],reverse=True)
    return sorted_class_nums[0][0]

#==ID3 algorithm===================
def createTree(dataSet,labels):
    label_list=[example[-1] for example in dataSet]
    if label_list.count(label_list[0])==len(label_list):
        return label_list[0]
    if len(dataSet[0])==1:
        return majorityCnt(label_list)
    best_feature=chooseBestFeatureToSplit(dataSet)
    best_label=labels[best_feature]
    my_tree={best_label:{}}
    feature_value=[example[best_feature] for example in dataSet]
    unique_value=set(feature_value)
    for value in unique_value:
        sublabels=labels[0:best_feature]
        sublabels.extend(labels[best_feature+1:])
        my_tree[best_label][value]=createTree(splitDataSet(dataSet, \
               best_feature, value),sublabels)
    return my_tree

labels=[]
for label in data.columns[1:][:-1]:
    labels.append(label)
data['density']=data['density'].apply(str)
data['sugar_ratio']=data['sugar_ratio'].apply(str)
retdata=data.iloc[:,1:]
dataSet=retdata.values.tolist()

entropy=calc_entropy(dataSet)
entropy

#选择最佳特征
best_feature=chooseBestFeatureToSplit(dataSet)
best_feature

my_tree=createTree(dataSet,labels)
my_tree

{'density': {'0.243': 0,
  '0.245': 0,
  '0.34299999999999997': 0,
  '0.36': 0,
  '0.40299999999999997': 1,
  '0.43700000000000006': 1,
  '0.48100000000000004': 1,
  '0.556': 1,
  '0.593': 0,
  '0.608': 1,
  '0.634': 1,
  '0.639': 0,
  '0.657': 0,
  '0.6659999999999999': 0,
  '0.6970000000000001': 1,
  '0.7190000000000001': 0,
  '0.774': 1}}